In [2]:
from loadTDMS import *
from fft_test import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns

In [3]:
def fft_integrated_one_rotation_fig(start,end,num,j):
    load_path = rf"Z:\100 Raw_data\20221027---切削実験\AE\Cutting\AE_{num}.tdms"
    d = load_tdms(load_path, ch_num=3)
    d = d[0][start:end]
    
    dt = 1/10**6
    split_t_r = 0.1 # 1つの枠で全体のどの割合のデータを分析するか。
    overlap = 0.5  # オーバーラップ率
    window_F = "hamming"  # 窓関数選択: hanning, hamming, blackman
    y_label = "amplitude"
    y_unit = "V"
    N = len(d)
    fq = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    
    
    fq_, amp_ = FFT_main(fq, d, dt, split_t_r, overlap, window_F)

    output_FN = fr"Z:\200 Produced_data\20221027\FFTonePath_Focus\fig\FFT_{num}_{x}"
    plot_FFT(fq_, amp_, output_FN, y_label, y_unit,num)
    
    return 20*np.log10(amp_) +85

## 各pathごとの面積のnpy生成

In [ ]:
for i in range(1,170):
    mensekis = []
    for j in tqdm(range(0,591)):
        num = i
        sample_num = 2**14
        start = j * 25000
        end = start + sample_num
        fft_result = fft_integrated_one_rotation_do_not_to_csv(start,end,num,j)
        menseki = time_hist_integrator_rect(1,fft_result)
        mensekis.append(menseki)
    d = np.array(mensekis)
    np.save(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\{num}", d)

## 各pathごとの面積の推移

In [ ]:
for num in tqdm(range(1,170)):
    fig = plt.figure()
    dd = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\{num}.npy")
    
    ax = fig.add_subplot(ylim=(11000,18000))

    plt.plot(dd)
    fig.savefig(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\推移\{num}.png")

## 各pathごとの面積histogram生成

In [ ]:
x_min = 10000
x_max = 18000
range_bin_width = range(x_min, x_max, 60)


for num in tqdm(range(1,170)):
    figure, ax = plt.subplots()
    dd = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\{num}.npy")
    hist_p = sns.distplot(dd, bins=range_bin_width, kde=False, rug=False)
    figure = hist_p.get_figure()
    hist_p.set_title(rf"{num}path")
    ax.set_ylim(0, 140)
    figure.savefig(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\{num}.png")

## 全pathの面積のheatmap

In [ ]:
dd3 = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\1.npy")
for num in tqdm(range(2,170)):
    dd2 = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\{num}.npy")
    dd3 = np.concatenate([dd3, dd2], axis=0)

dd4 = dd3.reshape(int(len(dd3)/591),591)

print(dd4.shape)

np.save(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\All", dd4)

## 全部の面積をプロット

In [ ]:
d = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\Aligned\All.npy")
d = d.ravel()

In [ ]:
%matplotlib qt

fig = plt.figure()

w_size = 591 * 10
v = np.ones(w_size) / w_size
d_moving_avg = np.convolve(d, v, mode='valid') 

ax = fig.add_subplot()
ax.set_xticks(np.arange(1, 169*591, 591*40))
ax.set_xticklabels(np.arange(0,170,40))

# plt.plot(d, lineWidth="2")
plt.plot(d_moving_avg, lineWidth="4")

## 周波数帯に区切って、面積出す

In [4]:
def get_tdms(num):
    load_path = rf"Z:\100 Raw_data\20221027---切削実験\AE\Cutting\AE_{num}.tdms"
    d = load_tdms(load_path, ch_num=3)
    return d[0]

In [ ]:
for i in range(1,170):
    d = get_tdms(i)
    mensekis = []
    
    for j in tqdm(range(0,591)):
        sample_num = 2**14
        start = j * 25000
        end = start + sample_num

        ds = d[start:end]
        fq_, amp_ = FFT_main(fq, ds, dt, split_t_r, overlap, window_F)
        amp_ = np.array(np.split(20*np.log10(amp_) +85, 10))

        menseki = np.sum(amp_, axis=1)
        mensekis.append(menseki)
    np.save(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\{i}", np.array(mensekis))           

In [ ]:
for i in range(0,10):
    d1 = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\1.npy")
    d1 = d1.T
    dd = d1[i]
#     print(dd.shape)
    for num in range(2,170):
        d2 = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\{num}.npy")
        d2 = d2.T
        dd = np.concatenate([dd,d2[i]])
#         print(dd.shape)
    start = i * 50
    end = (i+1) *50
    dd = dd.reshape(int(len(dd3)/591),591)
    print(dd.shape)
    np.save(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\All_{start}-{end}khz",dd)

## 各周波数帯の中で、正規化してばらつきが一番大きい周波数帯を特定

In [5]:
from sklearn import preprocessing

In [ ]:
d_min = []
d_max = []
spectrum = []
d_std = []

for i in range(0,10):
    start = i * 50
    end = (i+1) *50

    d = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\All_{start}-{end}khz.npy")
    d = d.T
    d = d.ravel()
    
    d_min.append(np.min(d))
    d_max.append(np.max(d))
    spectrum.append(f"{start}-{end}")
    
    d = preprocessing.minmax_scale(d)
    d_std.append(np.std(d))

d_min = np.array(d_min)
d_max = np.array(d_max)
d_minmax_diff = d_max - d_min


dict1 = dict(spectrum = spectrum[:] ,min_ = d_min[:], max_ = d_max[:], diff_ = d_minmax_diff[:], d_std = d_std[:])
df1 = pd.DataFrame(data=dict1)
df1.to_csv(rf'Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\EachSpectrumMinMax.csv')


In [ ]:
df = pd.read_csv(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\EachSpectrumMinMax.csv")
df

## 各周波数帯の中で正規化(normalization)し、heatmap表示

In [ ]:
for i in range(0,10):
    figure, ax = plt.subplots()
    start = i * 50
    end = (i+1) *50
    d = np.load(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\All_{start}-{end}khz.npy")
    d = d.ravel()
#     print("---------------------")
    d = preprocessing.minmax_scale(d)
#     print(len(d)/169)
    d = d.reshape(169, int(len(d)/169))
#     print(d.shape)
    np.save(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\AllNorm_{start}-{end}khz_test",d)
    sns.heatmap(d.T)
    figure.savefig(rf"Z:\200 Produced_data\20221027\FFTonePath_Focus\menseki\100khz\fig\NormT\AllNorm_{start}-{end}khz_heatmap_T_test.png", dpi=500)
